In [1]:
import numpy as np
import torch
import os
import pickle
import time
from Model import *
from itertools import chain
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.utils.data import DataLoader
from synthetic import simulate_lorenz_96, simulate_var
from utils import build_flags, time_split, save_result, evaluate_result, count_accuracy, loss_sparsity, loss_divergence, loss_mmd

In [2]:
def choseGPU():
    '''在程序运行最开始, 根据GPU使用情况，自动选择GPU'''
    from re import findall
    from subprocess import check_output
    from torch.cuda import set_device, is_available
    
    if(is_available()):
        output = check_output("nvidia-smi", shell=True)
        info = findall("(\d+)MiB\s/\s(\d+)MiB", output.decode("utf-8"))
        print("--> 开始查询GPU使用情况：")
        # 正确匹配
        best_index = 0
        current_um = -1
        for index, (um, am) in enumerate(info):
            print("第{:^3}块显卡：已使用{:^8}Mib, 全部{:^8}Mib，使用率:{:^8.2%}".format(index, um, am, float(um)/float(am)))
            best_index = index if(current_um>float(um) and current_um>=0) else best_index
            current_um = float(um)
        # 选择
        best_index=best_index
        print("--> 选择第{:^3}块GPU".format(best_index))
        set_device(best_index)
        
    else:
        print("显卡不可用")
        
# 设置随机数种子
# set_random_seed()
# 选择GPU
choseGPU()

--> 开始查询GPU使用情况：
第 0 块显卡：已使用 10808  Mib, 全部 24564  Mib，使用率: 44.00% 
第 1 块显卡：已使用   13   Mib, 全部 24564  Mib，使用率: 0.05%  
--> 选择第 1 块GPU


In [3]:
parser = build_flags()
args = parser.parse_args(args=[])
args.seed = 2
args.num_nodes = 264
args.dims = 1
args.threshold = 0.5
args.time_length = 500
args.time_step = 10
args.epochs = 3000
args.batch_size = 64
args.lr = 1e-3
args.weight_decay = 1e-3
args.encoder_alpha = 0.02
args.decoder_alpha = 0.02
args.beta_sparsity = 3.00 #0.25   #log_sum  #1.25是F=40,N=30的最优值
args.beta_kl = 0.01        #JS散度
args.beta_mmd = 2      #MMD
args.encoder_hidden = 20
args.decoder_hidden = 20
args.encoder_dropout = 0.1
args.decoder_dropout = 0.2

In [4]:
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

In [5]:
samples = np.load('/home/omnisky/Public/ChenRongfa/Intrer_VAE/PNC_SPM_emoid4.npy')
X_np = []
for sample in samples:
    X_np.append(np.array(time_split(sample[:,:,np.newaxis], step=10)))
X_np = np.vstack(X_np)
X_np = torch.FloatTensor(X_np)

In [6]:
print(X_np.shape)

torch.Size([41004, 264, 10, 1])


In [7]:
data = X_np
data_loader = DataLoader(data, batch_size=args.batch_size)

In [8]:
adj = []
for idx in range(264):
    decoder_file = 'decoder' + str(idx) + '.pt'
    decoder_file = os.path.join('/home/omnisky/Public/ChenRongfa/Intrer_VAE_result/DLCR_PNC_emoid4', decoder_file)
    decoder_net = decoder(args.dims, args.decoder_hidden, args.time_step - 1, args.num_nodes, args.decoder_dropout, args.decoder_alpha)
    decoder_net.load_state_dict(torch.load(decoder_file, map_location='cuda:1'))
    adj.append(decoder_net.adj[idx, :])

/home/omnisky/Public/ChenRongfa/Intrer_VAE/Model.py:20: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data, gain=1.414)


In [9]:
init_adj = torch.cat([temp.unsqueeze(0) for temp in adj], dim=0)
init_adj = init_adj.clone().detach()
print(init_adj.shape)

torch.Size([264, 264])


In [10]:
log_file = os.path.join('/home/omnisky/Public/ChenRongfa/Intrer_VAE_result/help4_3', 'log_val.txt')
log = open(log_file, 'w')
for idx in range(230,264):
    print('Begin training feature: {:04d}'.format(idx + 1))
    decoder_file = 'decoder' + str(idx) + '.pt'
    decoder_file = os.path.join('/home/omnisky/Public/ChenRongfa/Intrer_VAE_result/DLCR_PNC_emoid4', decoder_file)
    encoder_file = 'encoder' + str(idx) + '.pt'
    encoder_file = os.path.join('/home/omnisky/Public/ChenRongfa/Intrer_VAE_result/help4_3', encoder_file)

    Inter_decoder = decoder(args.dims, args.decoder_hidden, args.time_step - 1, args.num_nodes, args.decoder_dropout, args.decoder_alpha)
    Inter_decoder.load_state_dict(torch.load(decoder_file, map_location='cuda:1'))
    Inter_decoder = Inter_decoder.cuda()
    Inter_decoder.eval()

    Inter_encoder = encoder(init_adj, args.dims, args.encoder_hidden, args.dims, args.time_step - 1, args.encoder_dropout, args.encoder_alpha)
    Inter_encoder = Inter_encoder.cuda()

    optimizer = optim.Adam(Inter_encoder.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    loss_mse = nn.MSELoss()
    best_loss = np.inf
    
    for epoch in range(20):
        t = time.time()
        Loss = []
        MSE_loss = []
        SPA_loss = []
        KL_loss = []
        MMD_loss = []
        for batch_idx, data in enumerate(data_loader):
            optimizer.zero_grad()
            data = data.cuda()
            target = data[:, idx, 1:, :]
            inputs = data[:, :, :-1, :]

            mu, log_var = Inter_encoder(inputs)  #Inter_encoder(inputs, adj)
            sigma = torch.exp(log_var / 2)
            # sigma2 = torch.exp(log_var2 / 2)
            gamma = torch.randn(size = mu.size()).cuda()
            # theta = torch.randn(size = mu2.size()).cuda()
            gamma = mu + sigma * gamma
            # theta = mu2 + sigma2 * theta
            mask = torch.sigmoid(gamma) #* torch.sigmoid(theta) #* torch.sigmoid(theta + gamma)
            # gamma = torch.sigmoid(gamma)
            # theta = torch.sigmoid(theta)

            inputs = mask_inputs(mask, inputs)
            pred = Inter_decoder(inputs, idx)   #Inter_decoder(inputs, adj, idx)



            mse_loss = loss_mse(pred, target)
            spa_loss = loss_sparsity(mask, 'log_sum')
            kl_loss = loss_divergence(mask, 'JS')
            mmd_loss = loss_mmd(data[:, :, 1:, :], pred, idx)

            loss = mse_loss + args.beta_sparsity * spa_loss + args.beta_kl * kl_loss + args.beta_mmd * mmd_loss

            loss.backward()
            optimizer.step()

            Loss.append(loss.item())
            MSE_loss.append(mse_loss.item())
            SPA_loss.append(spa_loss.item())
            KL_loss.append(kl_loss.item())
            MMD_loss.append(mmd_loss.item())
        
        # if epoch == 500:
        #     optimizer.param_groups[0]['lr'] = args.lr/10

        if epoch % 5 == 0:
            print(  'Feature: {:04d}'.format(idx + 1),
                    'Epoch: {:04d}'.format(epoch),
                    'Loss: {:.10f}'.format(np.mean(Loss)),
                    'MSE_Loss: {:.10f}'.format(np.mean(MSE_loss)),
                    'Sparsity_loss: {:.10f}'.format(np.mean(SPA_loss)),
                    'KL_loss: {:.10f}'.format(np.mean(KL_loss)),
                    'MMD_loss: {:.10f}'.format(np.mean(MMD_loss)),
                    'time: {:.4f}s'.format(time.time() - t))
                
        if np.mean(Loss) < best_loss:
            best_loss = np.mean(Loss)
            #M[idx, :] = 
            # gamma_matrix[idx, :] = gamma.squeeze().mean(dim=2).mean(dim=1)
            # theta_matrix[idx, :] = theta.squeeze().mean(dim=2).mean(dim=1)
            # torch.save({
            #             'encoder_state_dict': Inter_encoder.state_dict(),
            #             'decoder_state_dict': Inter_decoder.state_dict(),
            #                 # 'adj' : adj

            #             }, encoder_file)
            torch.save(Inter_encoder.state_dict(), encoder_file)
                # np.save(save_file + str(idx) + '.npy', mask.cpu().detach().numpy())

            print('Feature: {:04d}'.format(idx + 1),
                  'Epoch: {:04d}'.format(epoch),
                  'Loss: {:.10f}'.format(np.mean(Loss)),
                  'mse_loss: {:.10f}'.format(np.mean(MSE_loss)),
                  'Sparsity_loss: {:.10f}'.format(np.mean(SPA_loss)),
                  'KL_loss: {:.10f}'.format(np.mean(KL_loss)),
                  'mmd_loss: {:.10f}'.format(np.mean(MMD_loss)),
                  'time: {:.4f}s'.format(time.time() - t), file=log)
        log.flush()
log.close()

Begin training feature: 0231
Feature: 0231 Epoch: 0000 Loss: 13.1691446877 MSE_Loss: 1.3565797349 Sparsity_loss: 3.9305438343 KL_loss: 0.0672202492 MMD_loss: 0.0101306271 time: 76.1780s
Feature: 0231 Epoch: 0005 Loss: 1.6860594985 MSE_Loss: 1.3084381302 Sparsity_loss: 0.1188597710 KL_loss: 0.0366788571 MMD_loss: 0.0103376261 time: 76.6707s
Feature: 0231 Epoch: 0010 Loss: 1.6869469199 MSE_Loss: 1.2928538629 Sparsity_loss: 0.1243957515 KL_loss: 0.0368710994 MMD_loss: 0.0102685409 time: 76.8215s
Feature: 0231 Epoch: 0015 Loss: 1.6869706588 MSE_Loss: 1.2929092171 Sparsity_loss: 0.1243849711 KL_loss: 0.0368710370 MMD_loss: 0.0102689088 time: 76.7585s
Begin training feature: 0232
Feature: 0232 Epoch: 0000 Loss: 12.0242579502 MSE_Loss: 1.4726261216 Sparsity_loss: 3.5075928746 KL_loss: 0.0465027150 MMD_loss: 0.0141940815 time: 76.8286s
Feature: 0232 Epoch: 0005 Loss: 1.6646925485 MSE_Loss: 1.4287137751 Sparsity_loss: 0.0689063295 KL_loss: 0.0351960881 MMD_loss: 0.0144539132 time: 76.8321s
Feat